# MAE 271: Lab 2 - Inverted pendulum with vertical base excitation
### Cooper Cook & Joshua Booth

## Introduction

## Code and figures
Define global parameters and initial conditions for the system:

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# -----------------------------
# Global Parameters
# -----------------------------
l_arm = 1.0             # length of pendulum arm [m]
m_mass = 1.0             # mass of pendulum mass [kg]
g = 9.8             # gravitational force [m/s^2]

# Time span
t_span = (0, 1)
t_eval = np.arange(0, 1.001, 0.001)

In [ ]:
# A = 2in bump height
for A in [2*0.0254]:   # Set bump height (convert in to m)
    results_A_2 = []  # store all simulation cases
    for n in [0,1]:
        # Nonlinear spring 
        G = (ks / (3 * (ms * g)**(2/3)))**3

        # Initial conditions: [ps, pus, qs, qt]
        if n == 1:
            initial = [0, 0, (ms * g / G)**(1/3), mt * g / kt]
        else:
            initial = [0, 0, 1.0 * ms * g / ks, mt * g / kt]

        for u in [20*0.46, 25*0.46, 30*0.46]:  # Loop through varying car speeds for each bump height (convert mph to m/s)

            # Model Function
            def LabDemoFunc(t, s):
                ps, pus, qs, qt = s

                # Road input
                X = u * t
                Y = 0.5 * A * (1 - np.cos(2 * np.pi * X / d))
                dYdX = 0.5 * A * (2 * np.pi / d) * np.sin(2 * np.pi * X / d)

                if X > d:
                    Y = 0
                    dYdX = 0

                vin = u * dYdX

                # Tire force
                Ft = kt * qt
                if qt <= 0:
                    Ft = 0

                # Velocities
                vs = ps / ms
                vus = pus / mus

                # Spring force
                if n == 1:
                    Fs = G * qs**3 + bs * (vus - vs)
                    Fss = G * qs**3
                else:
                    Fs = ks * qs + bs * (vus - vs)
                    Fss = ks * qs

                # State derivatives
                dps = -ms * g + Fs
                dpus = -mus * g + Ft - Fs
                dqs = vus - vs
                dqt = vin - vus

                ds = [dps, dpus, dqs, dqt]
                ext = [X, Y, Ft, vs, vus, vin, Fss]

                return ds, ext

            # Wrapper for solver
            def ode_wrapper(t, s):
                ds, _ = LabDemoFunc(t, s)
                return ds

            # -----------------------------
            # Run Simulation
            # -----------------------------
            sol = solve_ivp(ode_wrapper, t_span, initial, t_eval=t_eval)

            t = sol.t
            s = sol.y.T

            # Extract states
            ps, pus, qs, qt = s.T

            # Compute extra outputs
            ext = np.zeros((len(t), 7))
            ds = np.zeros((len(t), 4))

            for i in range(len(t)):
                ds[i], ext[i] = LabDemoFunc(t[i], s[i])

            dps, dpus, dqs, dqt = ds.T
            X, Y, Ft, vs, vus, vin, Fss = ext.T

            # Store results
            results_A_2.append({
                "A": A,
                "u": u,
                "t": t,
                "ps": ps,
                "pus": pus,
                "qs": qs,
                "qt": qt,
                "X": X,
                "Y": Y,
                "Ft": Ft,
                "vs": vs,
                "vus": vus,
                "vin": vin,
                "Fss": Fss,
                "dps": dps,
                "ms": ms,
                "lin": n
            })

    # Plot Results

    # Sprung masss acceleration
    plt.figure(figsize=(10,8))
    for r in results_A_2:
        label = f"A={r['A']:.2f} m, u={r['u']:.2f} m/s, non-linear = {r['lin']}"
        plt.plot(r["t"], (r["dps"]/r["ms"]), label=label)
        plt.ylabel("Acceleration [m/s^2]")
        plt.legend()

    plt.twinx()
    plt.plot(t, Y, label="Road", color="k")
    plt.ylabel("Road Input [m]")
    plt.xlabel("Time (s)")
    plt.ylim(0,0.10)
    plt.legend(loc='lower right')
    plt.title('Acceleration of Sprung Mass (A = 2in)')
    plt.grid(True)
    plt.show()

    # Relative displacement across the suspension
    plt.figure(figsize=(10,8))
    for r in results_A_2:
        label = f"A={r['A']:.2f} m, u={r['u']:.2f} m/s, non-linear = {r['lin']}"
        plt.plot(r["t"], (r["qs"]-r["qs"][0]), label=label)
        plt.ylabel("Displacement [m]")
        plt.legend()

    plt.twinx()
    plt.plot(t, Y, label="Road", color="k")
    plt.ylabel("Road Input [m]")
    plt.xlabel("Time (s)")
    plt.ylim(0,0.10)
    plt.legend(loc='lower right')
    plt.title('Relative displacement across the suspension (A = 2in)')
    plt.grid(True)
    plt.show()

    # Tire force
    plt.figure(figsize=(10,8))
    for r in results_A_2:
        label = f"A={r['A']:.2f} m, u={r['u']:.2f} m/s, non-linear = {r['lin']}"
        plt.plot(r["t"], (r["Ft"]), label=label)
        plt.ylabel("Tire Force [N]")
        plt.ylim(0,45000)
        plt.legend()

    plt.twinx()
    plt.plot(t, Y, label="Road", color="k")
    plt.ylabel("Road Input [m]")
    plt.xlabel("Time (s)")
    plt.ylim(0,0.10)
    plt.legend(loc='lower right')
    plt.title('Tire Force (A = 2in)')
    plt.grid(True)
    plt.show()